In [ ]:
pip install pandas fake_useragent requests_html lxml_html_clean

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from fake_useragent import UserAgent


In [30]:
first_url = 'Hotels-g186338-a_travelersChoice.1-London_England-Hotels.html'

# Generate the list of URLs using list comprehension
urls = [first_url] + [f"Hotels-g186338-oa{i:d}-a_travelersChoice.1-London_England-Hotels.html" for i in range(30, 3000, 30)]

urls

['Hotels-g186338-a_travelersChoice.1-London_England-Hotels.html',
 'Hotels-g186338-oa30-a_travelersChoice.1-London_England-Hotels.html',
 'Hotels-g186338-oa60-a_travelersChoice.1-London_England-Hotels.html',
 'Hotels-g186338-oa90-a_travelersChoice.1-London_England-Hotels.html',
 'Hotels-g186338-oa120-a_travelersChoice.1-London_England-Hotels.html',
 'Hotels-g186338-oa150-a_travelersChoice.1-London_England-Hotels.html',
 'Hotels-g186338-oa180-a_travelersChoice.1-London_England-Hotels.html',
 'Hotels-g186338-oa210-a_travelersChoice.1-London_England-Hotels.html',
 'Hotels-g186338-oa240-a_travelersChoice.1-London_England-Hotels.html',
 'Hotels-g186338-oa270-a_travelersChoice.1-London_England-Hotels.html',
 'Hotels-g186338-oa300-a_travelersChoice.1-London_England-Hotels.html',
 'Hotels-g186338-oa330-a_travelersChoice.1-London_England-Hotels.html',
 'Hotels-g186338-oa360-a_travelersChoice.1-London_England-Hotels.html',
 'Hotels-g186338-oa390-a_travelersChoice.1-London_England-Hotels.html',
 

In [31]:
import requests
from time import sleep
from urllib.parse import urljoin
import logging


class TripAdvisorScraper:
    def __init__(self, base_url='https://www.tripadvisor.com'):
        self.base_url = base_url
        self.session = requests.Session()
        # Using a realistic user agent
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5'
        }
        self.session.headers.update(self.headers)
        
        # Setup logging
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger('TripAdvisorScraper')

    def get_page_content(self, url_path, retry_count=3, delay=2):
        """
        Get HTML content from TripAdvisor pages with retry mechanism and rate limiting
        
        Args:
            url_path (str): The path or full URL to scrape
            retry_count (int): Number of retry attempts
            delay (int): Delay between requests in seconds
            
        Returns:
            str: HTML content of the page or None if failed
        """
        full_url = urljoin(self.base_url, url_path) if not url_path.startswith('http') else url_path
        
        for attempt in range(retry_count):
            try:
                # Respect rate limiting
                sleep(delay)
                
                response = self.session.get(full_url, timeout=10)
                response.raise_for_status()
                
                self.logger.info(f"Successfully retrieved content from {full_url}")
                return response.text

            except requests.exceptions.RequestException as e:
                self.logger.warning(f"Attempt {attempt + 1} failed: {str(e)}")
                if attempt == retry_count - 1:
                    self.logger.error(f"Failed to retrieve {full_url} after {retry_count} attempts")
                    return None
                sleep(delay * (attempt + 1))  # Exponential backoff

    def close(self):
        """Close the session"""
        self.session.close()


hotel_urls = []
hotel_names = []
def get_first_names(url):
    # Example usage
    scraper = TripAdvisorScraper()
    try:
        # Example: Get a hotel page
        content = scraper.get_page_content(url)
        if content:
            soup = BeautifulSoup(content, "html.parser")
            prop_box = soup.find_all('div', class_= "jsTLT K")

            for box in prop_box:
                if box.find('div', class_ = "nBrpc o W") is not None:
                    hotel_name = box.find('div', class_ = "nBrpc o W").text #.replace('\n\t\t','').strip()
                    hotel_names.append(hotel_name)
                else:
                    hotel_names.append('None')

                if box.find('a', attrs={"href": True}) is not None:
                    hotel_url = box.find('a', attrs={"href": True})   #.replace('\n\t\t','').replace('\n','').strip()
                    hotel_url = hotel_url['href']
                    hotel_urls.append(hotel_url)
                else:
                    hotel_urls.append('None')

            
            # Process the content as needed
    finally:
        scraper.close()

for i in urls:    
    get_first_names(i)

print(hotel_names)
print(hotel_urls)

INFO:TripAdvisorScraper:Successfully retrieved content from https://www.tripadvisor.com/Hotels-g186338-a_travelersChoice.1-London_England-Hotels.html
INFO:TripAdvisorScraper:Successfully retrieved content from https://www.tripadvisor.com/Hotels-g186338-oa30-a_travelersChoice.1-London_England-Hotels.html
INFO:TripAdvisorScraper:Successfully retrieved content from https://www.tripadvisor.com/Hotels-g186338-oa60-a_travelersChoice.1-London_England-Hotels.html
INFO:TripAdvisorScraper:Successfully retrieved content from https://www.tripadvisor.com/Hotels-g186338-oa90-a_travelersChoice.1-London_England-Hotels.html
INFO:TripAdvisorScraper:Successfully retrieved content from https://www.tripadvisor.com/Hotels-g186338-oa120-a_travelersChoice.1-London_England-Hotels.html
INFO:TripAdvisorScraper:Successfully retrieved content from https://www.tripadvisor.com/Hotels-g186338-oa150-a_travelersChoice.1-London_England-Hotels.html
INFO:TripAdvisorScraper:Successfully retrieved content from https://www.t

['1. Strand Palace', '2. The Resident Covent Garden', '3. Travelodge London Central Kings Cross', '4. Hilton London Metropole', '5. Canopy by Hilton London City', '6. Park Plaza London Riverbank', '7. Park Plaza London Park Royal', '8. Travelodge London Covent Garden', '9. Holiday Inn London - Kensington High St., an IHG hotel', '10. The Resident Soho', '11. The Clermont London, Charing Cross', '12. Park Plaza Westminster Bridge London', '13. The Westminster London, Curio Collection By Hilton', '14. Holiday Inn London - West, an IHG hotel', '15. The Royal Horseguards', '16. Hotel Riu Plaza London Victoria', '17. Montcalm Royal London House', '18. Travelodge London Docklands Central', '19. Wilde Aparthotels, London, Paddington', "20. The Bailey's Hotel London", '21. hub by Premier Inn London Westminster Abbey hotel', '22. The Montague on the Gardens', '23. Travelodge London Wembley High Road', '24. Novotel London Excel Hotel', '25. Sunborn London', '26. Palmers Lodge Swiss Cottage', '27

In [32]:
df = pd.DataFrame({
    'Hotel Name': hotel_names,
    'Hotel URL': hotel_urls
})

df

,Hotel Name,Hotel URL
0,1. Strand Palace,/Hotel_Review-g186338-d193112-Reviews-Strand_P...
1,2. The Resident Covent Garden,/Hotel_Review-g186338-d15134890-Reviews-The_Re...
2,3. Travelodge London Central Kings Cross,/Hotel_Review-g186338-d289706-Reviews-Travelod...
3,4. Hilton London Metropole,/Hotel_Review-g186338-d193089-Reviews-Hilton_L...
4,5. Canopy by Hilton London City,/Hotel_Review-g186338-d23246836-Reviews-Canopy...
...,...,...
2995,Villiers Tokyo Studio Apartments,/Hotel_Review-g186338-d3646100-Reviews-Villier...
2996,Wns Hotel Canary Wharf,/Hotel_Review-g186338-d23827490-Reviews-Wns_Ho...
2997,New Globe Hotel,/Hotel_Review-g186338-d7206821-Reviews-New_Glo...
2998,Piccadilly Circus Studios,/Hotel_Review-g186338-d9984092-Reviews-Piccadi...
